<a href="https://colab.research.google.com/github/pdomins/bayesian-learning/blob/master/ej3_bayesian_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [220]:
import pandas as pd
import numpy as np


In [221]:
df = pd.read_csv("3-binary(TP1).csv", delimiter=',', encoding='utf-8')
df

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


In [222]:
# Discretize GRE and GPA columns
df['gre_d'] = pd.cut(df['gre'], bins=[-float('inf'), 500, float('inf')], labels=['GRE < 500', 'GRE ≥ 500'])
df['gpa_d'] = pd.cut(df['gpa'], bins=[-float('inf'), 3, float('inf')], labels=['GPA < 3', 'GPA ≥ 3'])

print(df)

     admit  gre   gpa  rank      gre_d    gpa_d
0        0  380  3.61     3  GRE < 500  GPA ≥ 3
1        1  660  3.67     3  GRE ≥ 500  GPA ≥ 3
2        1  800  4.00     1  GRE ≥ 500  GPA ≥ 3
3        1  640  3.19     4  GRE ≥ 500  GPA ≥ 3
4        0  520  2.93     4  GRE ≥ 500  GPA < 3
..     ...  ...   ...   ...        ...      ...
395      0  620  4.00     2  GRE ≥ 500  GPA ≥ 3
396      0  560  3.04     3  GRE ≥ 500  GPA ≥ 3
397      0  460  2.63     2  GRE < 500  GPA < 3
398      0  700  3.65     2  GRE ≥ 500  GPA ≥ 3
399      0  600  3.89     3  GRE ≥ 500  GPA ≥ 3

[400 rows x 6 columns]


In [223]:
# Calculate probabilities
total_count = len(df)
prob_admit_0 = df['admit'].value_counts(normalize=True)[0]
prob_admit_1 = df['admit'].value_counts(normalize=True)[1]

In [224]:
def laplace_smooth_probs(arr):
    for index, row in arr.iterrows():
        total = sum(row)
        k = len(row)
        for i, val in row.items():
            arr.loc[index, i] = (val + 1) / (total + k)

    return arr

In [225]:
#sin Laplace
# conditional_probs = df.groupby(['gre_d', 'gpa_d', 'rank'])['admit'].value_counts(normalize=True).unstack().fillna(0)

#con Laplace
conditional_probs = laplace_smooth_probs(df.groupby(['gre_d', 'gpa_d', 'rank'])['admit'].value_counts().unstack().fillna(0))

conditional_probs

admit                          0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 1     0.714286  0.285714
                  2     0.545455  0.454545
                  3     0.928571  0.071429
                  4     0.900000  0.100000
          GPA ≥ 3 1     0.500000  0.500000
                  2     0.791667  0.208333
                  3     0.791667  0.208333
                  4     0.823529  0.176471
GRE ≥ 500 GPA < 3 1     0.200000  0.800000
                  2     0.761905  0.238095
                  3     0.545455  0.454545
                  4     0.625000  0.375000
          GPA ≥ 3 1     0.448980  0.551020
                  2     0.582524  0.417476
                  3     0.737500  0.262500
                  4     0.775000  0.225000

## a) Calcular la probabilidad de que una persona que proviene de una escuela con rango 1 no haya sido admitida en la universidad

In [226]:
# rank 1
conditional_probs_rank_1 = conditional_probs[conditional_probs.index.get_level_values('rank') == 1]
conditional_probs_rank_1

admit                          0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 1     0.714286  0.285714
          GPA ≥ 3 1     0.500000  0.500000
GRE ≥ 500 GPA < 3 1     0.200000  0.800000
          GPA ≥ 3 1     0.448980  0.551020

In [227]:
# P(admit = 0 | rank = 1) = P(admit = 0, rank = 1) / P(rank = 1)
#                         = sum(P(admit = 0, gre_d, gpa_d, rank = 1)) / sum(P(rank = 1, gre_d, gpa_d, admit))
#                           first column of conditional_probs_rank_1 / all values of conditional_probs_rank_1

probability = conditional_probs_rank_1.iloc[:, 0].sum() / conditional_probs_rank_1.sum().sum()
print(f'P(admit = 0 | rank = 1): {probability}')

P(admit = 0 | rank = 1): 0.4658163265306123


## b) Calcular la probabilidad de que una persona que fue a una escuela de rango 2, tenga GRE = 450 y GPA = 3.5 sea admitida en la universidad

In [228]:
# rank 2
conditional_probs_rank_2 = conditional_probs[conditional_probs.index.get_level_values('rank') == 2]
conditional_probs_rank_2

admit                          0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 2     0.545455  0.454545
          GPA ≥ 3 2     0.791667  0.208333
GRE ≥ 500 GPA < 3 2     0.761905  0.238095
          GPA ≥ 3 2     0.582524  0.417476

In [229]:
# P(admit = 1, gre_d = 450 , gpa_d = 3.5| rank = 2) => P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2)
#                                                    = P(admit = 1, gre_d < 500 , gpa_d ≥ 3) / P (rank = 2)
#                                                    = sum(P(admit = 1, gre_d < 500 , gpa_d ≥ 3, rank = 2)) / sum(P (rank = 2, admit, gre_d, gpa_d))

probability =  conditional_probs_rank_2.loc[('GRE < 500', 'GPA ≥ 3', 2), 1] / conditional_probs_rank_2.sum().sum()
print(f'P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2): {probability}')

P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2): 0.05208333333333334


### ❌ ❌ ❌ ❌ ❌

In [230]:
# P(admit = 1| rank = 2, GRE = 450 , GPA = 3.5) => P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3)
probability = conditional_probs_rank_2.loc[('GRE < 500', 'GPA ≥ 3', 2), 1]
print(f'P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3): {probability}')

P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3): 0.20833333333333334


### Tablas de Probabilidad Condicionales Asociadas

In [231]:
#Laplacian Smoothing
rank_probs = df['rank'].value_counts()
total = sum(rank_probs)
k = len(rank_probs)
for idx, prob in rank_probs.items():
    rank_probs[idx] = (prob + 1) / (total + k)

rank_probs = pd.DataFrame(rank_probs.sort_index(), columns=['rank'])
rank_probs.index.name = 'rank'

#No Laplacian Smoothing
rank_probs = df['rank'].value_counts(normalize=True)
rank_probs = pd.DataFrame(rank_probs.sort_index(), columns=['rank'])
rank_probs.index.name = 'rank'

rank_probs

,rank
rank,
1,0.1525
2,0.3775
3,0.3025
4,0.1675


In [232]:
#Laplacian Smoothing
rank_gpa = laplace_smooth_probs(df.groupby(['rank'])['gpa_d'].value_counts().unstack().fillna(0))

#No Laplacian Smoothing
rank_gpa = df.groupby(['rank'])['gpa_d'].value_counts(normalize=True).unstack().fillna(0)

rank_gpa

gpa_d,GPA ≥ 3,GPA < 3
rank,,
1,0.868852,0.131148
2,0.814570,0.185430
3,0.826446,0.173554
4,0.791045,0.208955


In [233]:
#Laplacian Smoothing
rank_gre = laplace_smooth_probs(df.groupby(['rank'])['gre_d'].value_counts().unstack().fillna(0))

#No Laplacian Smoothing
rank_gre = df.groupby(['rank'])['gre_d'].value_counts(normalize=True).unstack().fillna(0)

rank_gre

gre_d,GRE ≥ 500,GRE < 500
rank,,
1,0.819672,0.180328
2,0.794702,0.205298
3,0.719008,0.280992
4,0.656716,0.343284


In [234]:
#Laplacian Smoothing
admission_rank = laplace_smooth_probs(df.groupby(['rank'])['admit'].value_counts().unstack().fillna(0))

#No Laplacian Smoothing
admission_rank = df.groupby(['rank'])['admit'].value_counts(normalize=True).unstack().fillna(0)

admission_rank

admit,0,1
rank,,
1,0.459016,0.540984
2,0.642384,0.357616
3,0.768595,0.231405
4,0.820896,0.179104


In [235]:
#P(GRE, GPA, rank, admit) = P(GRE|rank) * P(GPA|rank) * P(admit|rank) * P(rank)
# needs tweaking, not quite right

def calculate_probabilities(rank_probs, rank_gpa, rank_gre, admission_rank):
    probabilities = []

    for gre_condition in ['GRE ≥ 500', 'GRE < 500']:
        for gpa_condition in ['GPA ≥ 3', 'GPA < 3']:
            for rank in rank_probs.index:
                p_rank = rank_probs.loc[rank, 'rank']

                p_gre_given_rank = rank_gre.loc[rank, gre_condition]
                p_gpa_given_rank = rank_gpa.loc[rank, gpa_condition]
                p_admit_given_rank_0 = admission_rank.loc[rank, 0]
                p_admit_given_rank_1 = admission_rank.loc[rank, 1]

                p_admit_and_rank_0 = p_gre_given_rank * p_gpa_given_rank * p_admit_given_rank_0 * p_rank
                p_admit_and_rank_1 = p_gre_given_rank * p_gpa_given_rank * p_admit_given_rank_1 * p_rank
                total = p_admit_and_rank_0 + p_admit_and_rank_1

                probabilities.append([gre_condition, gpa_condition, rank, p_admit_and_rank_0/total, p_admit_and_rank_1/total])

    result_df = pd.DataFrame(probabilities, columns=['gre_d', 'gpa_d', 'rank', 0, 1])
    return result_df

result = calculate_probabilities(rank_probs, rank_gpa, rank_gre, admission_rank)
# Group by the specified columns and select columns 0 and 1
grouped_result = result.groupby(['gre_d', 'gpa_d', 'rank'])[[0, 1]].sum()

grouped_result

0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 1     0.459016  0.540984
                  2     0.642384  0.357616
                  3     0.768595  0.231405
                  4     0.820896  0.179104
          GPA ≥ 3 1     0.459016  0.540984
                  2     0.642384  0.357616
                  3     0.768595  0.231405
                  4     0.820896  0.179104
GRE ≥ 500 GPA < 3 1     0.459016  0.540984
                  2     0.642384  0.357616
                  3     0.768595  0.231405
                  4     0.820896  0.179104
          GPA ≥ 3 1     0.459016  0.540984
                  2     0.642384  0.357616
                  3     0.768595  0.231405
                  4     0.820896  0.179104

In [236]:
# rank 1
conditional_probs_rank_1 = grouped_result[grouped_result.index.get_level_values('rank') == 1]
conditional_probs_rank_1

0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 1     0.459016  0.540984
          GPA ≥ 3 1     0.459016  0.540984
GRE ≥ 500 GPA < 3 1     0.459016  0.540984
          GPA ≥ 3 1     0.459016  0.540984

In [237]:
# P(admit = 0 | rank = 1) = P(admit = 0, rank = 1) / P(rank = 1)
#                         = sum(P(admit = 0, gre_d, gpa_d, rank = 1)) / sum(P(rank = 1, gre_d, gpa_d, admit))
#                           first column of conditional_probs_rank_1 / all values of conditional_probs_rank_1

probability = conditional_probs_rank_1.iloc[:, 0].sum() / conditional_probs_rank_1.sum().sum()
print(f'P(admit = 0 | rank = 1): {probability}')

P(admit = 0 | rank = 1): 0.45901639344262296


In [238]:
# rank 2
conditional_probs_rank_2 = grouped_result[grouped_result.index.get_level_values('rank') == 2]
conditional_probs_rank_2

0         1
gre_d     gpa_d   rank                    
GRE < 500 GPA < 3 2     0.642384  0.357616
          GPA ≥ 3 2     0.642384  0.357616
GRE ≥ 500 GPA < 3 2     0.642384  0.357616
          GPA ≥ 3 2     0.642384  0.357616

In [239]:
# P(admit = 1, gre_d = 450 , gpa_d = 3.5| rank = 2) => P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2)
#                                                    = P(admit = 1, gre_d < 500 , gpa_d ≥ 3) / P (rank = 2)
#                                                    = sum(P(admit = 1, gre_d < 500 , gpa_d ≥ 3, rank = 2)) / sum(P (rank = 2, admit, gre_d, gpa_d))

probability =  conditional_probs_rank_2.loc[('GRE < 500', 'GPA ≥ 3', 2), 1] / conditional_probs_rank_2.sum().sum()
print(f'P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2): {probability}')

P(admit = 1, gre_d < 500 , gpa_d ≥ 3 | rank = 2): 0.08940397350993379


In [240]:
# P(admit = 1| rank = 2, GRE = 450 , GPA = 3.5) => P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3)
probability = conditional_probs_rank_2.loc[('GRE < 500', 'GPA ≥ 3', 2), 1]
print(f'P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3): {probability}')

P(admit = 1 | rank = 2, GRE < 500 , GPA ≥ 3): 0.35761589403973515
